In [2]:
import pandas as pd
import re
import time
import datetime
import pickle
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegressionCV
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import numpy as np
#import scorecard_function 
#可能是作者自己的库

In [15]:
folderOfData = '/Users/zhijunwu/Desktop/A_Card/'

In [16]:
trainDataFile = open(folderOfData + 'trainData.pkl', 'rb')

In [17]:
import pprint, pickle

In [18]:
data1 = pickle.load(trainDataFile)
#print(data1)
pprint.pprint(data1)

       member_id  loan_amnt  term  loan_status int_rate emp_length  \
21001      21002       6250    36   Fully Paid    5.79%    2 years   
36324      36325      25000    36   Fully Paid   12.53%     1 year   
13517      13518       5000    36   Fully Paid   16.59%     1 year   
36635      36636      18000    36   Fully Paid   11.89%    2 years   
31632      31633       6000    36   Fully Paid   14.22%    2 years   
39463      39465       7500    36   Fully Paid    8.38%   < 1 year   
13195      13196       2800    36   Fully Paid   11.49%  10+ years   
11950      11951      15000    36  Charged Off   15.99%  10+ years   
4047        4048      30000    36   Fully Paid   14.27%    3 years   
22665      22666       5850    36   Fully Paid   15.20%  10+ years   
2557        2558       5000    36  Charged Off   15.96%    4 years   
36261      36262       6250    36   Fully Paid   13.47%    6 years   
37216      37217      13200    36   Fully Paid   13.47%    2 years   
16            17    

In [21]:
data1.shape

Index(['member_id', 'loan_amnt', 'term', 'loan_status', 'int_rate',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'total_acc',
       'pub_rec_bankruptcies', 'issue_d', 'earliest_cr_line', 'y'],
      dtype='object')

In [24]:
data1.columns

Index(['member_id', 'loan_amnt', 'term', 'loan_status', 'int_rate',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'total_acc',
       'pub_rec_bankruptcies', 'issue_d', 'earliest_cr_line', 'y'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)
data1.head()

,member_id,loan_amnt,term,loan_status,int_rate,emp_length,home_ownership,annual_inc,verification_status,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,pub_rec_bankruptcies,issue_d,earliest_cr_line,y
21001,21002,6250,36,Fully Paid,5.79%,2 years,MORTGAGE,46500.00,Verified,Borrower added on 01/10/11 > I wish to conso...,small_business,Paying off cards for good,195xx,PA,3.33,0,0,NaN,NaN,8,0,14,0.0,Jan-11,Sep-04,0
36324,36325,25000,36,Fully Paid,12.53%,1 year,RENT,235000.00,Verified,We are trying to buy our dream home and are a ...,house,Dream Home,606xx,IL,11.73,0,3,40.0,NaN,18,0,37,0.0,Jun-09,Aug-89,0
13517,13518,5000,36,Fully Paid,16.59%,1 year,RENT,38000.04,Source Verified,Borrower added on 06/10/11 > I would like to...,debt_consolidation,Bill Pay,201xx,VA,4.07,0,2,NaN,NaN,4,0,7,0.0,Jun-11,Jan-05,0
36635,36636,18000,36,Fully Paid,11.89%,2 years,RENT,87000.00,Not Verified,I would like to consolidate my credit card deb...,debt_consolidation,Consolidate credit card debt,220xx,VA,19.71,0,0,NaN,NaN,11,0,17,0.0,Jun-09,Dec-00,0
31632,31633,6000,36,Fully Paid,14.22%,2 years,OWN,50000.00,Not Verified,NaN,car,tonys car,630xx,MO,14.09,0,3,NaN,NaN,4,0,10,0.0,Mar-10,Jun-06,0


In [28]:
data1.emp_length.unique()


array(['2 years', '1 year', '< 1 year', '10+ years', '3 years', '4 years',
       '6 years', nan, '7 years', '5 years', '8 years', '9 years'],
      dtype=object)

In [42]:
def careerYear(x):
    #对工作年限处理
    x=str(x)
    if x.find('10+')>-1:
        return 11
    elif x.find('< 1')>-1:
        return 0
    else:
        return int(re.sub('\D','',x))


In [43]:
strr='1 year'
a=careerYear(strr)
a

1

In [48]:
def covertDateString(x):
    mth_dict={'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
                'Nov': 11, 'Dec': 12}
    if str(x)=='nan':
        return datetime.datetime.fromtimestamp(time.mktime(time.strptime('9900-1', '%Y-%m')))
    else:
        yr=int(x[4:6])
        if yr<=17:
            yr=2000+yr
        else:
            yr=1900+yr
        mth=mth_dict[x[:3]]
    return datetime.datetime(yr,mth,1)

In [46]:
datetime.datetime.fromtimestamp(time.mktime(time.strptime('9900-1','%Y-%m')))

datetime.datetime(9900, 1, 1, 0, 0)

In [49]:
data1['appl_date']=data1.issue_d.apply(lambda x: covertDateString(x))

In [51]:
data1

,member_id,loan_amnt,term,loan_status,int_rate,emp_length,home_ownership,annual_inc,verification_status,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,pub_rec_bankruptcies,issue_d,earliest_cr_line,y,appl_date
21001,21002,6250,36,Fully Paid,5.79%,2 years,MORTGAGE,46500.00,Verified,Borrower added on 01/10/11 > I wish to conso...,small_business,Paying off cards for good,195xx,PA,3.33,0,0,NaN,NaN,8,0,14,0.0,Jan-11,Sep-04,0,2011-01-01
36324,36325,25000,36,Fully Paid,12.53%,1 year,RENT,235000.00,Verified,We are trying to buy our dream home and are a ...,house,Dream Home,606xx,IL,11.73,0,3,40.0,NaN,18,0,37,0.0,Jun-09,Aug-89,0,2009-06-01
13517,13518,5000,36,Fully Paid,16.59%,1 year,RENT,38000.04,Source Verified,Borrower added on 06/10/11 > I would like to...,debt_consolidation,Bill Pay,201xx,VA,4.07,0,2,NaN,NaN,4,0,7,0.0,Jun-11,Jan-05,0,2011-06-01
36635,36636,18000,36,Fully Paid,11.89%,2 years,RENT,87000.00,Not Verified,I would like to consolidate my credit card deb...,debt_consolidation,Consolidate credit card debt,220xx,VA,19.71,0,0,NaN,NaN,11,0,17,0.0,Jun-09,Dec-00,0,2009-06-01
31632,31633,6000,36,Fully Paid,14.22%,2 years,OWN,50000.00,Not Verified,NaN,car,tonys car,630xx,MO,14.09,0,3,NaN,NaN,4,0,10,0.0,Mar-10,Jun-06,0,2010-03-01
39463,39465,7500,36,Fully Paid,8.38%,< 1 year,OWN,75000.00,Not Verified,Trying to further consolidate high interest co...,credit_card,High Interest Credit Consolidation,100xx,NY,6.06,0,0,0.0,0.0,7,0,9,NaN,Jan-08,May-99,0,2008-01-01
13195,13196,2800,36,Fully Paid,11.49%,10+ years,OWN,27560.00,Source Verified,NaN,other,Other Loan,880xx,NM,23.38,2,1,5.0,NaN,10,0,34,0.0,Jun-11,Aug-93,0,2011-06-01
11950,11951,15000,36,Charged Off,15.99%,10+ years,MORTGAGE,40000.00,Verified,Borrower added on 07/06/11 > Debt Consolidat...,debt_consolidation,Save my sanity,615xx,IL,16.92,0,1,56.0,NaN,23,0,36,0.0,Jul-11,Jan-94,1,2011-07-01
4047,4048,30000,36,Fully Paid,14.27%,3 years,MORTGAGE,60000.00,Verified,NaN,debt_consolidation,Debt Consolidation,262xx,WV,17.80,0,1,NaN,NaN,13,0,26,0.0,Nov-11,Oct-95,0,2011-11-01
22665,22666,5850,36,Fully Paid,15.20%,10+ years,RENT,23500.00,Source Verified,Borrower added on 12/06/10 > I am very relia...,car,Angie's Loan,928xx,CA,2.86,0,3,71.0,33.0,3,2,5,0.0,Dec-10,Mar-02,0,2010-12-01
